In [ ]:
import pandas as pd

In [26]:
# read and load the csv files
vol_bitfinex_by_supply_btc = pd.read_csv("light_data/vol_bitfinex_by_supply_btc.csv")
total_out_BTC_USD = pd.read_csv("light_data/total_out_BTC_USD.csv")

vol_bitfinex_by_supply_btc.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
total_out_BTC_USD.rename(columns={"Unnamed: 0": "Date"}, inplace=True)

In [32]:
vol_bitfinex_by_supply_btc
total_out_BTC_USD

,Date,total_out_BTC_usd
0,2013-03-31,8.506092e+05
1,2013-04-01,7.348477e+05
2,2013-04-02,1.545441e+06
3,2013-04-03,1.737993e+06
4,2013-04-04,1.967458e+06
...,...,...
3754,2023-07-11,1.835730e+08
3755,2023-07-12,1.975522e+08
3756,2023-07-13,2.349778e+08
3757,2023-07-14,2.077230e+08
